<h2>Promoting Tourism in Uzbekistan <h2>

We will use spatial and location data by Foursquare API to demonstrate venues first in Tashkent to show the data deficit, and then explore neighborhoods in New York and Toronto to locate the most common venue categories in each neighborhood.
We will then group the neighborhoods into clusters using the k-means clustering algorithm. We will visualize the neighborhoods in Toronto and New York City and their emerging clusters.
*The goal is to demonstrate to the Tourism Agency directors how Foursquare data could be used to promote tourism in each cities by identifying the most common venues, restaurants, parks, and etc.*


<h5>1) Install the necessary libraries<h5>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

**Enter credentials to get access to Foursquare developer account**

In [2]:
CLIENT_ID = 'AMMSPX1SBHCJVJ12LYUC1PE1AG2HCZXVFM3SR3LBNKBRYQYO' # your Foursquare ID
CLIENT_SECRET = 'UTAYXH20RCQOWU0P2G1KCLFQEE3A5I0XBJMGVGWSQSEBDOVT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AMMSPX1SBHCJVJ12LYUC1PE1AG2HCZXVFM3SR3LBNKBRYQYO
CLIENT_SECRET:UTAYXH20RCQOWU0P2G1KCLFQEE3A5I0XBJMGVGWSQSEBDOVT


Let's get the geographical coordinates of Tashkent.

In [3]:
address = 'Tashkent'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


41.3123363 69.2787079


**1. Search for a venue category**

In [62]:
search_query = 'Tashkent'
radius = 50000
print(search_query + ' .... OK!')

Tashkent .... OK!


Let's look for different venues

In [63]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=AMMSPX1SBHCJVJ12LYUC1PE1AG2HCZXVFM3SR3LBNKBRYQYO&client_secret=UTAYXH20RCQOWU0P2G1KCLFQEE3A5I0XBJMGVGWSQSEBDOVT&ll=41.3123363,69.2787079&v=20180604&query=Tashkent&radius=50000&limit=100'

In [64]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bf1bdff9fb6b75e315f4024'},
 'response': {'venues': [{'id': '4ee97e90aa1f7c7810245287',
    'name': 'Wyndham Hotel Tashkent',
    'location': {'address': 'Amir Temur Str. C 4 No. 7/8',
     'lat': 41.31726109460942,
     'lng': 69.28034140774784,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.31726109460942,
       'lng': 69.28034140774784}],
     'distance': 564,
     'postalCode': '100000',
     'cc': 'UZ',
     'city': 'Toshkent',
     'state': 'Toshkent Shahri',
     'country': 'Ўзбекистон',
     'formattedAddress': ['Amir Temur Str. C 4 No. 7/8',
      '100000 Toshkent',
      'Toshkent Shahri',
      'Ўзбекистон']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1542569471',
    'hasPerk': Fa

### Transform it into a *pandas* dataframe

In [65]:
#assign relevant part of JSON to venues
venues = results['response']['venues']

## transform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ee97e90aa1f7c7810245287,Amir Temur Str. C 4 No. 7/8,UZ,Toshkent,Ўзбекистон,NaN,564,"[Amir Temur Str. C 4 No. 7/8, 100000 Toshkent,...","[{'label': 'display', 'lat': 41.31726109460942...",41.317261,69.280341,100000,Toshkent Shahri,Wyndham Hotel Tashkent,v-1542569471,NaN
1,"[{'id': '50aa9e094b90af0d42d5de0d', 'name': 'C...",False,4e6342641838ad3d0f27b37d,NaN,UZ,NaN,Ўзбекистон,NaN,544,"[100000, Toshkent Shahri, Ўзбекистон]","[{'label': 'display', 'lat': 41.30751019348326...",41.307510,69.279754,100000,Toshkent Shahri,Ташкент | Tashkent (Toshkent),v-1542569471,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,56bae6af498e75fd3201de84,Navoi ave,UZ,Toshkent,Ўзбекистон,NaN,449,"[Navoi ave, Toshkent, Toshkent Shahri, Ўзбекис...","[{'label': 'display', 'lat': 41.31633651719231...",41.316337,69.277935,NaN,Toshkent Shahri,HYATT Regency Hotel Tashkent,v-1542569471,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4c6b935de503c928846448b3,"ул. Буюк Турон, 56",UZ,Toshkent,Ўзбекистон,NaN,874,"[ул. Буюк Турон, 56, 100029 Toshkent, Toshkent...","[{'label': 'display', 'lat': 41.30997536562405...",41.309975,69.268735,100029,Toshkent Shahri,Lotte City Hotel Tashkent Palace,v-1542569471,69803467
4,"[{'id': '4bf58dd8d48988d1a6941735', 'name': 'L...",False,4f1fa1a4e4b0a6dc4f908105,NaN,UZ,Toshkent,Ўзбекистон,Amir Temur,52,"[Amir Temur, Toshkent, Toshkent Shahri, Ўзбеки...","[{'label': 'display', 'lat': 41.31216049369441...",41.312160,69.278124,NaN,Toshkent Shahri,Tashkent State University of Law | Ташкентский...,v-1542569471,NaN


### Focus on the interest and filter dataframe

In [66]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Wyndham Hotel Tashkent,Hotel,Amir Temur Str. C 4 No. 7/8,UZ,Toshkent,Ўзбекистон,NaN,564,"[Amir Temur Str. C 4 No. 7/8, 100000 Toshkent,...","[{'label': 'display', 'lat': 41.31726109460942...",41.317261,69.280341,100000,Toshkent Shahri,4ee97e90aa1f7c7810245287
1,Ташкент | Tashkent (Toshkent),City,NaN,UZ,NaN,Ўзбекистон,NaN,544,"[100000, Toshkent Shahri, Ўзбекистон]","[{'label': 'display', 'lat': 41.30751019348326...",41.307510,69.279754,100000,Toshkent Shahri,4e6342641838ad3d0f27b37d
2,HYATT Regency Hotel Tashkent,Hotel,Navoi ave,UZ,Toshkent,Ўзбекистон,NaN,449,"[Navoi ave, Toshkent, Toshkent Shahri, Ўзбекис...","[{'label': 'display', 'lat': 41.31633651719231...",41.316337,69.277935,NaN,Toshkent Shahri,56bae6af498e75fd3201de84
3,Lotte City Hotel Tashkent Palace,Hotel,"ул. Буюк Турон, 56",UZ,Toshkent,Ўзбекистон,NaN,874,"[ул. Буюк Турон, 56, 100029 Toshkent, Toshkent...","[{'label': 'display', 'lat': 41.30997536562405...",41.309975,69.268735,100029,Toshkent Shahri,4c6b935de503c928846448b3
4,Tashkent State University of Law | Ташкентский...,Law School,NaN,UZ,Toshkent,Ўзбекистон,Amir Temur,52,"[Amir Temur, Toshkent, Toshkent Shahri, Ўзбеки...","[{'label': 'display', 'lat': 41.31216049369441...",41.312160,69.278124,NaN,Toshkent Shahri,4f1fa1a4e4b0a6dc4f908105
5,Türkiye Cumhuriyeti Tashkent Büyükelçiliği,Embassy / Consulate,NaN,UZ,NaN,Ўзбекистон,NaN,977,[Ўзбекистон],"[{'label': 'display', 'lat': 41.30617024862404...",41.306170,69.287023,NaN,NaN,50c1a33b498e5a9cfe881121
6,Цветы Ташкента | Tashkent Flowers,Flower Shop,Bukhara Str,UZ,Tashkent,Ўзбекистон,NaN,656,"[Bukhara Str, Tashkent, UZS, Ўзбекистон]","[{'label': 'display', 'lat': 41.3100154022778,...",41.310015,69.271493,NaN,UZS,4f952034e4b0d0d98a2958d4
7,Ж/Д Вокзал Ташкент | Tashkent Railway Station,Train Station,Uzbekiston Ovozi St,UZ,Tashkent,Ўзбекистон,NaN,2432,"[Uzbekiston Ovozi St, Tashkent, Toshkent Viloy...","[{'label': 'display', 'lat': 41.291383, 'lng':...",41.291383,69.286963,NaN,Toshkent Viloyati,4c9b79769c663704509061fd
8,Tashkent Aeroport 3 Terminal,Airport Terminal,Sergeli District,UZ,Toshkent,Ўзбекистон,NaN,6039,"[Sergeli District, Toshkent, Toshkent Shahri, ...","[{'label': 'display', 'lat': 41.25967280227663...",41.259673,69.261338,NaN,Toshkent Shahri,4e92662db8032b150acb32e4
9,Toshkent Xalqaro Aeroporti | Tashkent Internat...,Airport,Sergeli District,UZ,Toshkent,Ўзбекистон,NaN,5669,"[Sergeli District, Toshkent, Toshkent Shahri, ...","[{'label': 'display', 'lat': 41.26219476451237...",41.262195,69.266825,NaN,Toshkent Shahri,4ddac10f2271ac9047645224


**Get the names of the venues**

In [67]:
dataframe_filtered.name

0                                Wyndham Hotel Tashkent
1                         Ташкент | Tashkent (Toshkent)
2                          HYATT Regency Hotel Tashkent
3                      Lotte City Hotel Tashkent Palace
4     Tashkent State University of Law | Ташкентский...
5            Türkiye Cumhuriyeti Tashkent Büyükelçiliği
6                     Цветы Ташкента | Tashkent Flowers
7         Ж/Д Вокзал Ташкент | Tashkent Railway Station
8                          Tashkent Aeroport 3 Terminal
9     Toshkent Xalqaro Aeroporti | Tashkent Internat...
10                Tashkent TV Tower | Телебашня Ташкент
11                           Tashkent Kargo (Aviakassa)
12                                      CJ ICM Tashkent
13                            Hotel Uzbekistan Tashkent
14    Ташкентский Финансовый Институт/Tashkent Finan...
15                         VIP Lounge, Tashkent Airport
16                                 Tashkent Clock Tower
17      Tashkent University of Information Techn

In [69]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=5,
    color='red',
    popup='Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Let's find the next 100 venues around the city

In [83]:
LIMIT=1000
radius=15000
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    radius, 
    LIMIT)
url2

'https://api.foursquare.com/v2/venues/explore?&client_id=AMMSPX1SBHCJVJ12LYUC1PE1AG2HCZXVFM3SR3LBNKBRYQYO&client_secret=UTAYXH20RCQOWU0P2G1KCLFQEE3A5I0XBJMGVGWSQSEBDOVT&v=20180604&ll=41.3123363,69.2787079&radius=15000&limit=1000'

In [84]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5bf1c0341ed21966bb4e261d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Tashkent',
  'headerFullLocation': 'Tashkent',
  'headerLocationGranularity': 'city',
  'totalResults': 170,
  'suggestedBounds': {'ne': {'lat': 41.44733643500013,
    'lng': 69.4581036202576},
   'sw': {'lat': 41.177336164999865, 'lng': 69.0993121797424}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56968d73498ea26925905f67',
       'name': 'HORI japanese & Pan-Asia restaurant and lounge',
       'location': {'lat': 41.312227519393005,
        'lng': 69.28346321774839,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.312227519393005,
          'ln

In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [88]:
venues = results2['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,HORI japanese & Pan-Asia restaurant and lounge,Seafood Restaurant,41.312228,69.283463
1,Bon!,Coffee Shop,41.305257,69.273145
2,Ilkhom Theatre / Театр Ильхом,Theater,41.318169,69.261366
3,Площадь и памятник Амира Темура | Amir Timur S...,Park,41.311082,69.279699
4,"Кофейня ""ЧайКоф""",Café,41.296244,69.280828


In [89]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<h3> Demonstration of venues in NYC <h3>

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.
The data is available by running a `wget` command and access the data.

In [90]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [91]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [93]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [94]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [95]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [96]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [97]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [98]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [99]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [100]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [101]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [102]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [103]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [104]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

Get the neighborhood's latitude and longitude values.

In [105]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [127]:
# type your answer here
LIMIT = 100
radius = 5000

urlNY = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
urlNY 



'https://api.foursquare.com/v2/venues/explore?&client_id=AMMSPX1SBHCJVJ12LYUC1PE1AG2HCZXVFM3SR3LBNKBRYQYO&client_secret=UTAYXH20RCQOWU0P2G1KCLFQEE3A5I0XBJMGVGWSQSEBDOVT&v=20180604&ll=40.87655077879964,-73.91065965862981&radius=5000&limit=100'

Send the GET request and examine the resutls

In [128]:
results = requests.get(urlNY).json()
results

{'meta': {'code': 200, 'requestId': '5bf1c5f9f594df7ee30daa52'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'New York',
  'headerFullLocation': 'New York',
  'headerLocationGranularity': 'city',
  'totalResults': 240,
  'suggestedBounds': {'ne': {'lat': 40.921550823799684,
    'lng': -73.85125646411224},
   'sw': {'lat': 40.831550733799595, 'lng': -73.97006285314738}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4baf59e8f964a520a6f93be3',
       'name': 'Bikram Yoga',
       'location': {'address': '5500 Broadway',
        'crossStreet': '230th Street',
        'lat': 40.876843690797934,
        'lng': -73.90620384419528,
        'lab

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [129]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [130]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Sam's Pizza,Pizza Place,40.879435,-73.905859
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Arturo's,Pizza Place,40.874412,-73.910271
4,Indian Road Café,Café,40.872922,-73.918459


And how many venues were returned by Foursquare?

In [131]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [133]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(urlNY).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [134]:
# type your answer here
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'])




Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


#### Let's check the size of the resulting dataframe

In [140]:
print(manhattan_venues.shape)
manhattan_venues.head()

(4000, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Sam's Pizza,40.879435,-73.905859,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
4,Marble Hill,40.876551,-73.91066,Indian Road Café,40.872922,-73.918459,Café


Let's check how many venues were returned for each neighborhood

In [141]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,100,100,100,100,100,100
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,100,100,100,100,100,100
East Village,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [142]:
list(manhattan_venues)

['Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'Venue',
 'Venue Latitude',
 'Venue Longitude',
 'Venue Category']

In [139]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 56 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [143]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Athletics & Sports,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Bistro,Botanical Garden,Burger Joint,Café,Caribbean Restaurant,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Farmers Market,Frozen Yogurt Shop,Garden,Golf Course,Gourmet Shop,Greek Restaurant,History Museum,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Museum,Park,Pet Store,Pizza Place,Playground,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Spanish Restaurant,Supermarket,Tapas Restaurant,Track,Trail,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [144]:
manhattan_onehot.shape

(4000, 57)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [145]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Athletics & Sports,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Bistro,Botanical Garden,Burger Joint,Café,Caribbean Restaurant,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Farmers Market,Frozen Yogurt Shop,Garden,Golf Course,Gourmet Shop,Greek Restaurant,History Museum,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Museum,Park,Pet Store,Pizza Place,Playground,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Spanish Restaurant,Supermarket,Tapas Restaurant,Track,Trail,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Yoga Studio
0,Battery Park City,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
1,Carnegie Hill,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2,Central Harlem,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
3,Chelsea,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
4,Chinatown,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
5,Civic Center,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
6,Clinton,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
7,East Harlem,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
8,East Village,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
9,Financial District,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.04,0.01,0.03,0.03,0.01,0.01,0.05,0.01,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.05,0.01,0.02,0.01,0.01,0.05,0.01,0.06,0.01,0.09,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


#### Let's confirm the new size

In [146]:
manhattan_grouped.shape

(40, 57)

#### Let's print each neighborhood along with the top 5 most common venues

In [147]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                       venue  freq
0                Pizza Place  0.09
1                       Park  0.06
2         Mexican Restaurant  0.05
3                      Diner  0.05
4  Latin American Restaurant  0.05


----Carnegie Hill----
                       venue  freq
0                Pizza Place  0.09
1                       Park  0.06
2         Mexican Restaurant  0.05
3                      Diner  0.05
4  Latin American Restaurant  0.05


----Central Harlem----
                       venue  freq
0                Pizza Place  0.09
1                       Park  0.06
2         Mexican Restaurant  0.05
3                      Diner  0.05
4  Latin American Restaurant  0.05


----Chelsea----
                       venue  freq
0                Pizza Place  0.09
1                       Park  0.06
2         Mexican Restaurant  0.05
3                      Diner  0.05
4  Latin American Restaurant  0.05


----Chinatown----
                       venue  freq
0           

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [148]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [150]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
1,Carnegie Hill,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
2,Central Harlem,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
3,Chelsea,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
4,Chinatown,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant


<a id='item4'></a>

## 4. Cluster Neighborhoods

## Introduction

There are many models for **clustering** out there. In this notebook, we will be presenting the model that is considered one of the simplest models amongst them. Despite its simplicity, the **K-means** is vastly used for clustering in many data science applications, especially useful if you need to quickly discover insights from **unlabeled data**. In this notebook, we will use k-Means for clustering venues.

Some real-world applications of k-means:
- Customer segmentation
- Understand what the visitors of a website are trying to accomplish
- Pattern recognition
- Machine learning
- Data compression

Run *k*-means to cluster the neighborhood into 5 clusters.

In [152]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [153]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant


Finally, let's visualize the resulting clusters

In [154]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [156]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
1,Chinatown,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
2,Washington Heights,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
3,Inwood,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
4,Hamilton Heights,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
5,Manhattanville,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
6,Central Harlem,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
7,East Harlem,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
8,Upper East Side,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant
9,Yorkville,Pizza Place,Park,Latin American Restaurant,Mexican Restaurant,Diner,Garden,Café,Coffee Shop,Deli / Bodega,Japanese Restaurant


#### Cluster 2

In [162]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 3

In [161]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 4

In [159]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 5

In [160]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


<h3> Demonstration of venues in Toronto <h3>

1) Installing the required libraries...

In [163]:
!pip install msgpack
!pip install url
!pip install wikipedia

    100% |████████████████████████████████| 317kB 18.4MB/s 
    100% |████████████████████████████████| 143kB 16.7MB/s 
  Running setup.py bdist_wheel for url ... done
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/98/c1/fe/9281ec4e03517032e1d6e51082c75ea5fa737dacba55627286
Successfully built url
  Running setup.py bdist_wheel for wikipedia ... done
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [164]:
import wikipedia as wp
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
df = pd.read_html(url)[0]
df.to_csv('beautifulsoup_pandas.csv',header=0,index=True)
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


3) Create a dataframe from the csv file. 

In [165]:
df_boroughs = pd.read_csv('beautifulsoup_pandas.csv')
df_boroughs.head()

,0,Postcode,Borough,Neighbourhood
0,1,M1A,Not assigned,Not assigned
1,2,M2A,Not assigned,Not assigned
2,3,M3A,North York,Parkwoods
3,4,M4A,North York,Victoria Village
4,5,M5A,Downtown Toronto,Harbourfront


4) Let's see the shape of the dataframe whether there are enough boroughs and neighborhoods

In [166]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_boroughs['Borough'].unique()),
        df_boroughs.shape[0]
    )
)

The dataframe has 12 boroughs and 289 neighborhoods.


5) Let's drop the "Not assigned" boroughs

In [167]:
df_boroughs_new = df_boroughs[df_boroughs.Borough != "Not assigned"]
df_boroughs_new.reset_index(drop=True)
df_boroughs_new.head()

,0,Postcode,Borough,Neighbourhood
2,3,M3A,North York,Parkwoods
3,4,M4A,North York,Victoria Village
4,5,M5A,Downtown Toronto,Harbourfront
5,6,M5A,Downtown Toronto,Regent Park
6,7,M6A,North York,Lawrence Heights


6) Let's check the new df_boroughs_new dataframe

In [168]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_boroughs_new['Borough'].unique()),
        df_boroughs_new.shape[0]
    )
)

The dataframe has 11 boroughs and 212 neighborhoods.


Now it's time to replace the "Not assigned" values in "Neighbourhood" column with the Borough name

In [169]:
df_boroughs_new[df_boroughs_new.Neighbourhood == 'Not assigned']
df_boroughs_new.Neighbourhood[8]= "Queen's Park"

df_boroughs_new

df_boroughs_new.reset_index(drop=True)



/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,Postcode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M5A,Downtown Toronto,Regent Park
4,7,M6A,North York,Lawrence Heights
5,8,M6A,North York,Lawrence Manor
6,9,M7A,Queen's Park,Queen's Park
7,11,M9A,Etobicoke,Islington Avenue
8,12,M1B,Scarborough,Rouge
9,13,M1B,Scarborough,Malvern


In [170]:
df_boroughs_new.groupby('Postcode').count()
df_boroughs_new.groupby('Postcode')["Neighbourhood"].sum()
df_boroughs_new.groupby('Postcode')["Neighbourhood"].apply(list)
df_boroughs_new.groupby('Postcode')["Neighbourhood"].apply(set)
dfm  = df_boroughs_new.groupby(['Postcode', 'Borough'])["Neighbourhood"].agg(lambda col: ', '.join(col)).reset_index()
dfm

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [171]:
'The number of rows of the dataframe is {}'.format(dfm.shape[0])

'The number of rows of the dataframe is 103'

In [172]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
geodata = geodata[['Latitude','Longitude']]
geodata.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


We will combine the postal code dataframe with the geodata dataframe

In [173]:
canada_postcode_df = pd.concat([dfm, geodata], axis=1)

canada_postcode_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [174]:
borough = canada_postcode_df.copy()
borough['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [175]:
selected_borough = borough[borough['Borough']=="North York"]
#print('Selecting the {}'.format(borough.shape[0]))
selected_borough.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
19,M2K,North York,Bayview Village,43.786947,-79.385975
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [176]:
b_name = 'North York'
b_latitude = selected_borough.loc[17, 'Latitude']
b_longitude = selected_borough.loc[17, 'Longitude']

#n_latitude = Scarborough.loc[8, 'Latitude']
#n_longitude = Scarborough.loc[8, 'Longitude']

print('Latitude and longitude values of {} are {}, {}.'.format(b_name, 
                                                               b_latitude, 
                                                               b_longitude))

Latitude and longitude values of North York are 43.8037622, -79.3634517.


In [178]:
import json # library to handle JSON files
LIMIT = 100
radius = 1500
urlTNT = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    b_latitude, 
    b_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=AMMSPX1SBHCJVJ12LYUC1PE1AG2HCZXVFM3SR3LBNKBRYQYO&client_secret=UTAYXH20RCQOWU0P2G1KCLFQEE3A5I0XBJMGVGWSQSEBDOVT&v=20180604&ll=43.8037622,-79.3634517&radius=1500&limit=100'

In [180]:
results = requests.get(urlTNT).json()

In [181]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [182]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) #flatten json
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('There are {} Venues aroung the Selected Borough  returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

There are 47 Venues aroung the Selected Borough  returned by Foursquare.


,name,categories,lat,lng
0,Tastee,Bakery,43.807722,-79.356798
1,Bayview Golf & Country Club,Golf Course,43.809391,-79.375285
2,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,Korean Restaurant,43.798391,-79.369187
3,Chatime Willowdale,Bubble Tea Shop,43.791326,-79.367506
4,Galati,Grocery Store,43.797831,-79.369410


In [183]:
venues_list = nearby_venues.groupby('categories').count()
venues_list.sort_values('name', ascending=False, inplace=True)
venues_list.reset_index(inplace=True)
venues_list.head(4)

,categories,name,lat,lng
0,Coffee Shop,6,6,6
1,Chinese Restaurant,5,5,5
2,Park,4,4,4
3,Pharmacy,3,3,3


In [184]:
favor1 = venues_list.loc[venues_list.index[0],'categories']
favor2 = venues_list.loc[venues_list.index[1],'categories']
print(favor1 + ' and  ' + favor2)

Coffee Shop and  Chinese Restaurant


<h4> Chinese Restaurant and Coffee shops are the most popular venues in the neighborhood. <h4>

In [185]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[b_latitude, b_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(selected_borough['Latitude'], selected_borough['Longitude'], selected_borough['Borough'], selected_borough['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto